In [ ]:
%load_ext autoreload
%autoreload 2
from ecodonut.eco_frame import EcoFrameCalculator, mark_territory, get_influencers_in_point, EcoFrame, LayerOptions,merge_ecoframes
import geopandas as gpd
import pickle


## Создание эко-каркаса с нуля

In [ ]:
# Чтение исходных геоданных (ЖД, свалки, природные резервы, промышленные зоны, лесные массивы и водные объекты) и формирование словаря слоёв

layers = {
    # 'industrial': gpd.read_parquet('example_data/industrial_lo_spb.parquet'), # Индастриал добавим позже для демонстрации работы соеднения каркасов
    "gas_station": gpd.read_parquet('example_data/gas_station_lo_spb.parquet'),
    "landfill": gpd.read_parquet('example_data/landfill_lo_spb.parquet'),
    'regional_roads': gpd.read_parquet('example_data/regional_roads_lo_spb.parquet'),
    'federal_roads': gpd.read_parquet('example_data/federal_roads_lo_spb.parquet'),
    "nature_reserve": gpd.read_parquet('example_data/nature_reserve_lo_spb.parquet'),
    'woods': gpd.read_parquet('example_data/woods_lo_spb.parquet'),
    "water": gpd.read_parquet('example_data/water_lo_spb.parquet'),

    "railway": gpd.GeoDataFrame(),
}
# Загрузка границ региона
borders = gpd.read_file('example_data/lo_bounds.geojson')

# Создание экологического каркаса с заданием границ
eco_frame: EcoFrame = EcoFrameCalculator(territory=borders).evaluate_ecoframe(layers, min_layer_count=2,
                                                                              max_layer_count=8)


In [ ]:
# Сохранение объекта EcoFrame в файл с помощью pickle
pickle.dump(eco_frame, open('eco_frame_spb.pickle', 'wb'))

In [ ]:

# Загрузка сохранённого объекта EcoFrame из файла
eco_frame: EcoFrame = pickle.load(open('eco_frame_spb.pickle', 'rb'))

In [ ]:
# Сохранение GeoDataFrame экокаркаса в .gpkg
eco_frame.eco_frame.to_file('eco_frame_full.gpkg')

### Оценка территории

In [ ]:
# Загрузка зоны для оценки территории
project_zone = gpd.read_file('example_data/project Шлиссельбург.geojson')

# Оценка территории
mark = mark_territory(eco_frame, project_zone)
mark

### Объекты в точке(геометрии)

In [ ]:
from shapely import Point

point = gpd.GeoDataFrame(geometry=[Point(352932.23, 6643489.22)], crs=32636)
influencers = get_influencers_in_point(eco_frame, point, around_buffer=5)
influencers

# Создание эко-каркаса с настройками существующего и их соедиение

In [ ]:
layers = {
    'industrial': gpd.read_parquet('example_data/industrial_lo_spb.parquet'), # Индастриал добавим позже для демонстрации работы соеднения каркасов
}
borders = gpd.read_file('example_data/lo_bounds.geojson')

eco_frame_only_industrial: EcoFrame = EcoFrameCalculator(territory=borders,settings_from=eco_frame).evaluate_ecoframe(layers, min_layer_count=2,
                                                                              max_layer_count=8)

### Соединение двух каркасов в рамках территории

In [ ]:
project_zone = gpd.read_file('example_data/project Шлиссельбург.geojson')

# merged_ecoframe:EcoFrame = merge_ecoframes(eco_frame,eco_frame_only_industrial) # merge two ecoframes at all

merged_ecoframe:EcoFrame = merge_ecoframes(eco_frame,eco_frame_only_industrial,project_zone)

In [ ]:
merged_ecoframe.eco_frame.to_file('eco_frame_full_merge.gpkg')

## Антропогенное воздействие по почве от предприятий с классом опасности

In [ ]:


industrial = gpd.read_parquet('example_data/industrial_lo_spb.parquet')
layers = {
    'industrial': industrial.iloc[[1]],
}


def _industrial_danglvl_to_init(dang_lvl: int):
    init_values = {1: -10, 2: -8, 3: -6, 4: -4}
    return init_values.get(dang_lvl, -2)


def _industrial_danglvl_to_fading(dang_lvl: int):
    fading_values = {1: 0.2, 2: 0.1, 3: 0.05, 4: 0.01}
    return fading_values.get(dang_lvl, 0.005)


def _calculate_impact(impact_list: list, min_value, **kwargs) -> float:
    if len(impact_list) == 1:
        return impact_list[0]
    negative_list = sorted([abs(x) for x in impact_list if x < 0])
    total_negative = min(min(negative_list), abs(min_value))
    return - total_negative


borders = gpd.read_file('example_data/lo_bounds.geojson')
layers_options = {
    "industrial": LayerOptions(
        initial_impact=("dangerous_level", _industrial_danglvl_to_init),
        fading=("dangerous_level", _industrial_danglvl_to_fading),
        russian_name="Промышленный объект",
    )
}
# Создание потенциального распространения загрязнения по почве от ПОО 
frame: EcoFrame = EcoFrameCalculator(territory=borders, layer_options=layers_options,
                                     impact_calculator=_calculate_impact).evaluate_ecoframe(layers, min_layer_count=3,
                                                                                            max_layer_count=6)
